In [7]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.utils.colab.set_colab_file_id('1-0HlBq7rU-2nLmwOS8iHpN0Ib8CNCK5L')

# Insurance cost prediction using linear regression

Make a submisson here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from [Kaggle](https://www.kaggle.com/mirichoi0218/insurance).


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model





In [8]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

In [9]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [10]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [11]:
DATASET_URL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [12]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [13]:
your_name = 'Pranav' # at least 5 characters

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [14]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [15]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,region,charges
240,23,female,41.8038,2,yes,northeast,37356.279451
651,53,female,45.1440,1,no,southeast,10262.319670
761,23,male,40.1280,1,no,southwest,2344.446350
836,36,male,35.9100,0,no,southwest,4270.166010
1147,20,female,36.3888,0,no,northwest,2193.721736


Let us answer some basic questions about the dataset. 


**Q: How many rows does the dataset have?**

In [16]:
num_rows = dataframe.shape[0]
print(num_rows)

1271


**Q: How many columns doe the dataset have**

In [17]:
num_cols = dataframe.shape[1]
print(num_cols)

7


**Q: What are the column titles of the input variables?**

In [18]:
input_cols = ['age','sex','bmi','smoker','children']

**Q: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [19]:
categorical_cols = ['sex','smoker','region']

**Q: What are the column titles of output/target variable(s)?**

In [20]:
output_cols = ['charges']

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ml/aakashns/dataviz-cheatsheet

In [21]:
print("Minimum value of the charges column is {} ".format(dataframe['charges'].min()))
print("Maximum value of the charges column is {} ".format(dataframe['charges'].max()))
print("Average value of the charges column is {} ".format(dataframe['charges'].mean()))# Write your answer here

Minimum value of the charges column is 1088.217683 
Maximum value of the charges column is 61857.3151697 
Average value of the charges column is 12972.222715690816 


## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [32]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [34]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[23.    ,  0.    , 41.8038,  1.    ,  2.    ],
        [53.    ,  0.    , 45.144 ,  0.    ,  1.    ],
        [23.    ,  1.    , 40.128 ,  0.    ,  1.    ],
        ...,
        [59.    ,  0.    , 41.9121,  1.    ,  1.    ],
        [53.    ,  0.    , 37.905 ,  0.    ,  0.    ],
        [48.    ,  0.    , 37.9962,  0.    ,  0.    ]]),
 array([[37356.279451 ],
        [10262.31967  ],
        [ 2344.44635  ],
        ...,
        [46459.8876095],
        [10247.937965 ],
        [ 8035.170279 ]]))

**Q: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [36]:
inputs = torch.from_numpy(inputs_array).float()
targets = torch.from_numpy(targets_array).float()

In [37]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [39]:
dataset = TensorDataset(inputs, targets)

**Q: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [40]:
val_percent = 0.15 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split(dataset,[train_size,val_size]) # Use the random_split function to split dataset into 2 parts of the desired length

Finally, we can create data loaders for training & validation.

**Q: Pick a batch size for the data loader.**

In [41]:
batch_size = 12

In [42]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [43]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[40.0000,  1.0000, 28.5912,  0.0000,  0.0000],
        [27.0000,  1.0000, 52.3260,  0.0000,  2.0000],
        [42.0000,  0.0000, 30.3240,  1.0000,  0.0000],
        [24.0000,  0.0000, 31.6008,  0.0000,  0.0000],
        [34.0000,  1.0000, 24.3675,  0.0000,  0.0000],
        [39.0000,  1.0000, 33.7440,  0.0000,  4.0000],
        [44.0000,  0.0000, 31.3500,  0.0000,  1.0000],
        [47.0000,  1.0000, 44.3916,  1.0000,  2.0000],
        [38.0000,  1.0000, 19.1691,  0.0000,  2.0000],
        [29.0000,  0.0000, 44.2662,  0.0000,  3.0000],
        [59.0000,  1.0000, 33.8580,  0.0000,  2.0000],
        [46.0000,  1.0000, 48.2790,  1.0000,  3.0000]])
targets: tensor([[ 5253.1914],
        [ 3582.6252],
        [20708.2441],
        [ 2390.6802],
        [ 4365.3291],
        [ 7286.8989],
        [ 7398.1831],
        [42876.5742],
        [ 6441.3286],
        [ 4984.1089],
        [12538.1094],
        [44766.5898]])


Let's save our work by committing to Jovian.

In [44]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ··········
[jovian] Uploading colab notebook to Jovian...
[jovian] Committed successfully! https://jovian.ai/pranavrnambiar/02-insurance-linear-regression


'https://jovian.ai/pranavrnambiar/02-insurance-linear-regression'

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [47]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [48]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)              # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                        # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(out, targets)                        # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)                          # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [52]:
model = InsuranceModel()
model

InsuranceModel(
  (linear): Linear(in_features=5, out_features=1, bias=True)
)

Let's check out the weights and biases of the model using `model.parameters`.

In [50]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.3715, -0.1492, -0.0865,  0.2777, -0.2649]], requires_grad=True),
 Parameter containing:
 tensor([0.2380], requires_grad=True)]

One final commit before we train the model.

In [51]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Committed successfully! https://jovian.ai/pranavrnambiar/02-insurance-linear-regression


'https://jovian.ai/pranavrnambiar/02-insurance-linear-regression'

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [56]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [58]:
result =evaluate(model, val_loader)  # Use the the evaluate function
print(result)

{'val_loss': 12782.7705078125}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [59]:
epochs = 1000
lr = 1e-1
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6546.7388
Epoch [40], val_loss: 6515.2285
Epoch [60], val_loss: 6515.0977
Epoch [80], val_loss: 6507.5952
Epoch [100], val_loss: 6547.8931
Epoch [120], val_loss: 6526.0581
Epoch [140], val_loss: 6451.8608
Epoch [160], val_loss: 6440.4165
Epoch [180], val_loss: 6443.3413
Epoch [200], val_loss: 6447.9771
Epoch [220], val_loss: 6414.0420
Epoch [240], val_loss: 6421.2275
Epoch [260], val_loss: 6388.2705
Epoch [280], val_loss: 6456.0591
Epoch [300], val_loss: 6409.4023
Epoch [320], val_loss: 6388.5254
Epoch [340], val_loss: 6353.7988
Epoch [360], val_loss: 6376.8511
Epoch [380], val_loss: 6359.0591
Epoch [400], val_loss: 6333.4658
Epoch [420], val_loss: 6422.1748
Epoch [440], val_loss: 6331.6948
Epoch [460], val_loss: 6383.9531
Epoch [480], val_loss: 6297.5566
Epoch [500], val_loss: 6293.7788
Epoch [520], val_loss: 6279.5698
Epoch [540], val_loss: 6294.1235
Epoch [560], val_loss: 6307.8506
Epoch [580], val_loss: 6268.8496
Epoch [600], val_loss: 6289.6479
Epoch [620], v

In [60]:
epochs = 1000
lr = 1e-1
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6090.7080
Epoch [40], val_loss: 6091.3638
Epoch [60], val_loss: 6095.1812
Epoch [80], val_loss: 6127.3960
Epoch [100], val_loss: 6075.9019
Epoch [120], val_loss: 6064.5273
Epoch [140], val_loss: 6066.3193
Epoch [160], val_loss: 6104.9072
Epoch [180], val_loss: 6076.8945
Epoch [200], val_loss: 6046.6714
Epoch [220], val_loss: 6105.1235
Epoch [240], val_loss: 6034.8052
Epoch [260], val_loss: 6032.4116
Epoch [280], val_loss: 6063.8213
Epoch [300], val_loss: 5994.8496
Epoch [320], val_loss: 6018.0513
Epoch [340], val_loss: 6057.6226
Epoch [360], val_loss: 6022.4863
Epoch [380], val_loss: 5974.3999
Epoch [400], val_loss: 5980.8506
Epoch [420], val_loss: 5949.5532
Epoch [440], val_loss: 5945.2339
Epoch [460], val_loss: 5970.0488
Epoch [480], val_loss: 5968.4297
Epoch [500], val_loss: 5969.4341
Epoch [520], val_loss: 5934.6240
Epoch [540], val_loss: 5995.5879
Epoch [560], val_loss: 5982.9248
Epoch [580], val_loss: 5888.9717
Epoch [600], val_loss: 5987.0625
Epoch [620], v

In [61]:
epochs = 1000
lr = 1e-1
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 5737.9316
Epoch [40], val_loss: 5770.2319
Epoch [60], val_loss: 5753.6670
Epoch [80], val_loss: 5725.5527
Epoch [100], val_loss: 5726.7432
Epoch [120], val_loss: 5696.3438
Epoch [140], val_loss: 5800.8135
Epoch [160], val_loss: 5681.1284
Epoch [180], val_loss: 5696.4678
Epoch [200], val_loss: 5713.6787
Epoch [220], val_loss: 5716.0322
Epoch [240], val_loss: 5707.9092
Epoch [260], val_loss: 5721.5952
Epoch [280], val_loss: 5700.5820
Epoch [300], val_loss: 5634.4082
Epoch [320], val_loss: 5687.9238
Epoch [340], val_loss: 5666.3032
Epoch [360], val_loss: 5610.8779
Epoch [380], val_loss: 5657.7905
Epoch [400], val_loss: 5607.4336
Epoch [420], val_loss: 5639.6113
Epoch [440], val_loss: 5677.1855
Epoch [460], val_loss: 5574.3848
Epoch [480], val_loss: 5592.6016
Epoch [500], val_loss: 5590.5151
Epoch [520], val_loss: 5607.9150
Epoch [540], val_loss: 5640.0835
Epoch [560], val_loss: 5574.0591
Epoch [580], val_loss: 5533.4893
Epoch [600], val_loss: 5531.3301
Epoch [620], v

In [62]:
epochs = 1000
lr = 1e-1
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 5399.3936
Epoch [40], val_loss: 5409.7080
Epoch [60], val_loss: 5444.1909
Epoch [80], val_loss: 5451.5898
Epoch [100], val_loss: 5382.2617
Epoch [120], val_loss: 5360.3052
Epoch [140], val_loss: 5339.8857
Epoch [160], val_loss: 5361.3843
Epoch [180], val_loss: 5336.1724
Epoch [200], val_loss: 5324.9502
Epoch [220], val_loss: 5360.1299
Epoch [240], val_loss: 5358.5259
Epoch [260], val_loss: 5334.1152
Epoch [280], val_loss: 5317.4766
Epoch [300], val_loss: 5283.4175
Epoch [320], val_loss: 5322.8950
Epoch [340], val_loss: 5327.1479
Epoch [360], val_loss: 5307.3516
Epoch [380], val_loss: 5258.2061
Epoch [400], val_loss: 5253.1396
Epoch [420], val_loss: 5240.8848
Epoch [440], val_loss: 5247.6138
Epoch [460], val_loss: 5258.6162
Epoch [480], val_loss: 5240.6733
Epoch [500], val_loss: 5224.7012
Epoch [520], val_loss: 5246.5918
Epoch [540], val_loss: 5206.6782
Epoch [560], val_loss: 5232.0996
Epoch [580], val_loss: 5275.0557
Epoch [600], val_loss: 5226.0034
Epoch [620], v

In [63]:
epochs = 1000
lr = 1e-1
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 5066.7856
Epoch [40], val_loss: 5044.1899
Epoch [60], val_loss: 5025.3115
Epoch [80], val_loss: 5017.9927
Epoch [100], val_loss: 5045.3696
Epoch [120], val_loss: 5022.1558
Epoch [140], val_loss: 4993.0361
Epoch [160], val_loss: 5005.4336
Epoch [180], val_loss: 5051.6250
Epoch [200], val_loss: 5025.7031
Epoch [220], val_loss: 5034.7397
Epoch [240], val_loss: 4986.3325
Epoch [260], val_loss: 4997.8833
Epoch [280], val_loss: 4953.6406
Epoch [300], val_loss: 4986.3496
Epoch [320], val_loss: 4954.9814
Epoch [340], val_loss: 4984.5630
Epoch [360], val_loss: 4928.4287
Epoch [380], val_loss: 4933.9556
Epoch [400], val_loss: 4904.2544
Epoch [420], val_loss: 4948.0239
Epoch [440], val_loss: 4940.2212
Epoch [460], val_loss: 4886.7266
Epoch [480], val_loss: 4894.2144
Epoch [500], val_loss: 4875.5381
Epoch [520], val_loss: 4869.4609
Epoch [540], val_loss: 4861.7954
Epoch [560], val_loss: 4951.7095
Epoch [580], val_loss: 4909.1055
Epoch [600], val_loss: 4846.2178
Epoch [620], v

**Q: What is the final validation loss of your model?**

In [64]:
val_loss = [history5[-1]]
val_loss

[{'val_loss': 4751.7119140625}]

Let's log the final validation loss to Jovian and commit the notebook

In [65]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Metrics logged.


In [66]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/pranavrnambiar/02-insurance-linear-regression


'https://jovian.ai/pranavrnambiar/02-insurance-linear-regression'

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q: Complete the following function definition to make predictions on a single input**

In [68]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)          # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [69]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([32.0000,  1.0000, 40.1280,  0.0000,  2.0000])
Target: tensor([4530.5210])
Prediction: tensor([4888.6709])


In [70]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([46.0000,  1.0000, 34.7643,  1.0000,  3.0000])
Target: tensor([39498.9336])
Prediction: tensor([18529.5176])


In [71]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([42.0000,  1.0000, 35.6307,  0.0000,  0.0000])
Target: tensor([6168.0132])
Prediction: tensor([6785.8125])


In [72]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/pranavrnambiar/02-insurance-linear-regression
[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/pranavrnambiar/02-insurance-linear-regression


'https://jovian.ai/pranavrnambiar/02-insurance-linear-regression'